In [1]:
import mmlspark

if mmlspark.__spark_package_version__ < "1.0.0-rc3":
    raise Exception("This notebook is not compatible with the current version of mmlspark: {}. Please upgrade to 1.0.0-rc3 or higher.".format(
        mmlspark.__spark_package_version__))

In [2]:
from mmlspark.cognitive import *
from notebookutils import mssparkutils
from pyspark.sql.functions import explode

In [3]:
# Fetch the subscription key (or a general Cognitive Service key) from Azure Key Vault
service_key = mssparkutils.credentials.getSecret("keyvault-AzureML", "CogServiceTextSecret", "LinktoKeyVault")

In [4]:
# Load the data into a Spark DataFrame
df = spark.sql("SELECT * FROM default.customercomments")

sentiment = (TextSentiment()
    .setLocation("westus2")
    .setSubscriptionKey(service_key)
    .setOutputCol("output")
    .setErrorCol("error")
    .setLanguage("en")
    .setTextCol("comment"))

results = sentiment.transform(df)

# Show the results
display(results\
    .select("comment", explode("output").alias("exploded"), "error")\
    .select("comment", "exploded.*", "error")\
    .limit(10))

In [5]:
df.write.mode("overwrite").saveAsTable("SentimentAnalysis")